Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.848891
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.835130
Minibatch accuracy: 25.0%
Validation accuracy: 52.5%
Minibatch loss at step 100: 1.055205
Minibatch accuracy: 75.0%
Validation accuracy: 70.5%
Minibatch loss at step 150: 0.408395
Minibatch accuracy: 93.8%
Validation accuracy: 73.6%
Minibatch loss at step 200: 0.380762
Minibatch accuracy: 87.5%
Validation accuracy: 77.3%
Minibatch loss at step 250: 0.715822
Minibatch accuracy: 75.0%
Validation accuracy: 74.4%
Minibatch loss at step 300: 0.902715
Minibatch accuracy: 68.8%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.552933
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.755430
Minibatch accuracy: 75.0%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.514883
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.581931
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [21]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        max_ = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                             padding='SAME')
        conv = tf.nn.conv2d(max_, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        max_ = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                             padding='SAME')
        shape = max_.get_shape().as_list()
        reshape = tf.reshape(max_, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [19]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.663912
Minibatch accuracy: 12.5%
Validation accuracy: 10.7%
Minibatch loss at step 50: 2.006258
Minibatch accuracy: 25.0%
Validation accuracy: 40.6%
Minibatch loss at step 100: 1.058923
Minibatch accuracy: 75.0%
Validation accuracy: 67.8%
Minibatch loss at step 150: 0.440444
Minibatch accuracy: 87.5%
Validation accuracy: 74.5%
Minibatch loss at step 200: 0.515179
Minibatch accuracy: 75.0%
Validation accuracy: 78.1%
Minibatch loss at step 250: 0.458297
Minibatch accuracy: 87.5%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.783579
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 350: 0.742238
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.677381
Minibatch accuracy: 75.0%
Validation accuracy: 82.0%
Minibatch loss at step 450: 0.707234
Minibatch accuracy: 93.8%
Validation accuracy: 82.5%
Minibatch loss at step 500: 0.374889
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [67]:
learn_rate = 0.01
batch_size = 16
momentum = 0.8

patch1_size = 5
depth1 = 6

max_pool1_patch = 2
max_pool1_stride = 2

patch2_size = 3
depth2 = 12

max_pool2_patch = 2
max_pool2_stride = 2

dense_hidden1 = 60

dense_hidden2 = 30

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch1_size, patch1_size, num_channels, depth1], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth1]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch2_size, patch2_size, depth1, depth2], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // (max_pool1_stride) // (max_pool2_stride) * \
             image_size // (max_pool1_stride) // (max_pool2_stride) * depth2, dense_hidden1], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[dense_hidden1]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
            [dense_hidden1, dense_hidden2], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[dense_hidden2]))
    
    layer5_weights = tf.Variable(tf.truncated_normal(
            [dense_hidden2, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        max_ = tf.nn.max_pool(hidden, ksize=[1, max_pool1_patch, max_pool1_patch, 1], 
                              strides=[1, max_pool1_stride, max_pool1_stride, 1], padding='SAME')
        conv = tf.nn.conv2d(max_, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        max_ = tf.nn.max_pool(hidden, ksize=[1, max_pool2_patch, max_pool2_patch, 1], 
                              strides=[1, max_pool2_stride, max_pool2_stride, 1], padding='SAME')
        shape = max_.get_shape().as_list()
        reshape = tf.reshape(max_, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        return tf.matmul(hidden, layer5_weights) + layer5_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.MomentumOptimizer(learn_rate, momentum).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [68]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print(' - Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.903817
Minibatch accuracy: 6.2%
 - Validation accuracy: 9.8%
Minibatch loss at step 100: 2.265683
Minibatch accuracy: 18.8%
 - Validation accuracy: 10.0%
Minibatch loss at step 200: 1.866566
Minibatch accuracy: 43.8%
 - Validation accuracy: 29.2%
Minibatch loss at step 300: 1.281577
Minibatch accuracy: 43.8%
 - Validation accuracy: 59.6%
Minibatch loss at step 400: 0.815598
Minibatch accuracy: 68.8%
 - Validation accuracy: 74.4%
Minibatch loss at step 500: 0.642494
Minibatch accuracy: 87.5%
 - Validation accuracy: 76.2%
Minibatch loss at step 600: 0.942161
Minibatch accuracy: 75.0%
 - Validation accuracy: 78.2%
Minibatch loss at step 700: 0.790991
Minibatch accuracy: 81.2%
 - Validation accuracy: 79.9%
Minibatch loss at step 800: 0.739741
Minibatch accuracy: 81.2%
 - Validation accuracy: 80.7%
Minibatch loss at step 900: 0.731200
Minibatch accuracy: 81.2%
 - Validation accuracy: 82.6%
Minibatch loss at step 1000: 0.618399
Minibatch accuracy: 81.2

In [5]:
def run_conv_layer(hidden, weights, bias, stride=1, padding='SAME'):
    conv = tf.nn.conv2d(hidden, weights, [1, stride, stride, 1], padding='SAME')
    hidden = tf.nn.relu(conv + bias)
    return hidden

In [6]:
def inception_module(hidden, pool_1b1_weights, one_by_one_weights,
                     both_onebyone_5b5_weights, both_onebyone_3b3_weights):
    
    one_by_one_5b5_weights, five_by_five_weights = both_onebyone_5b5_weights
    one_by_one_3b3_weights, three_by_three_weights = both_onebyone_3b3_weights
    
    avg_pooled = tf.nn.avg_pool(hidden, [1, 5, 5, 1], [1, 1, 1, 1], padding='SAME')
    pool_1b1 = run_conv_layer(avg_pooled, *list(pool_1b1_weights))
    
    just_1b1 = run_conv_layer(hidden, *list(one_by_one_weights))
    
    after1b1 = run_conv_layer(hidden, *list(one_by_one_3b3_weights))
    one_by_one_3b3 = run_conv_layer(after1b1, *list(three_by_three_weights))
    
    after1b1 = run_conv_layer(hidden, *list(one_by_one_5b5_weights))
    one_by_one_5b5 = run_conv_layer(after1b1, *list(five_by_five_weights))
#     tf.nn.conv2d(avg_pooled, pool_1b1_weights, [1, 1, 1, 1], padding='SAME') 
    
#     just_1b1 = tf.nn.conv2d(hidden, one_by_one_weights)
    
#     after1b1 = tf.nn.conv2d(hidden, one_by_one_3b3_weights, [1, 1, 1, 1], padding='SAME')
#     one_by_one_3b3 = tf.nn.conv2d(after1b1, three_by_three_weights, [1, 1, 1, 1], padding='SAME')
    
#     after1b1 = tf.nn.conv2d(hidden, one_by_one_5b5_weights, [1, 1, 1, 1], padding='SAME')
#     one_by_one_5b5 = tf.nn.conv2d(after1b1, five_by_five_weights, [1, 1, 1, 1], padding='SAME')
    
    concatenated = tf.concat(3, [pool_1b1, just_1b1, one_by_one_3b3, one_by_one_5b5])
    return concatenated    

In [7]:
def init_dense_weights(in_filter_size, out_filter_size, std=0.1, bias=1.0):
    weight = tf.Variable(tf.truncated_normal([in_filter_size, out_filter_size], stddev=0.1))
    bias = tf.Variable(tf.constant(bias, shape=[out_filter_size]))
    return (weight, bias)

In [8]:
def init_patch_weights(patch_size, in_filter_size, out_filter_size, std=0.1, bias=1.0):
    weight = tf.Variable(tf.truncated_normal([patch_size, patch_size, in_filter_size, out_filter_size], stddev=std))
    bias = tf.Variable(tf.constant(bias, shape=[out_filter_size]))
    return (weight, bias)

In [9]:
def init_inception_weights(in_size, pool_11_size, one_size, three_sizes, five_sizes):
    pool_weights = init_patch_weights(1, in_size, pool_11_size)
    one_weights = init_patch_weights(1, in_size, one_size)
    three_weights1 = init_patch_weights(3, in_size, three_sizes[0])
    three_weights2 = init_patch_weights(1, three_sizes[0], three_sizes[1])
    five_weights1 = init_patch_weights(5, in_size, five_sizes[0])
    five_weights2 = init_patch_weights(5, five_sizes[0], five_sizes[1])
    return pool_weights, one_weights, (three_weights1, three_weights2), (five_weights1, five_weights2)    

In [ ]:
learn_rate = 0.01
batch_size = 200
momentum = 0.6

# Layer 0: (N x 28 x 28 x 1)

# Layer 1: (N x 28 x 28 x 16)
patch1_size = 5
depth1 = 16

# Layer 2: (N x 14 x 14 x 16)
max_pool1_patch = 2
max_pool1_stride = 2

# Layer 3: (N x 14 x 14 x 64)
inception_sizes0 = [8, 8, (12, 16), (12, 16)]
inception_depth0 = 48
# patch2_size = 3
# depth2 = 64

# Layer 4: (N x 14 x 14 x 32)
inception_sizes = [8, 8, (12, 8), (12, 8)]
inception_depth = 32

# Layer 5: (N x 7 x 7 x 32)
max_pool2_patch = 2
max_pool2_stride = 2

# Layer 6: (N x 7 x 7 x 8) -> (N x 392)
patch3_size = 1
depth3 = 8

# Layer 7: (N x 128)
dense_hidden1 = 128

# Layer 8: (N x 64)
dense_hidden2 = 64

# Layer 9: (N x 10)

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset[:2000])
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
#     p11c = tf.Variable(tf.truncated_normal(
#         [1, 1, hidden.get_]))
    
    layer1_weights, layer1_biases = init_patch_weights(patch1_size, 1, depth1, std=0.25, bias=1.0)
    
#     layer2_weights, layer2_biases = init_patch_weights(patch2_size, depth1, depth2)
    inc0_weights = init_inception_weights(depth1, *inception_sizes0)
    
    inc1_weights = init_inception_weights(inception_depth0, *inception_sizes)
    
    layer3_weights, layer3_biases = init_patch_weights(1, inception_depth, depth3, std=0.25)
    
    layer4_weights, layer4_biases = init_dense_weights(7*7*depth3, dense_hidden1, bias=1.0)
    
    layer5_weights, layer5_biases = init_dense_weights(dense_hidden1, dense_hidden2, bias=1.0)
    
    layer6_weights, layer6_biases = init_dense_weights(dense_hidden2, num_labels, bias=1.0)
    
    
    # Model.
    def model(data):
        out1 = run_conv_layer(data, layer1_weights, layer1_biases)
        max_ = tf.nn.max_pool(out1, ksize=[1, max_pool1_patch, max_pool1_patch, 1], 
                              strides=[1, max_pool1_stride, max_pool1_stride, 1], padding='SAME')
#         out2 = run_conv_layer(max_, layer2_weights, layer2_biases)
        out2 = inception_module(max_, *inc0_weights)
        
        out3 = inception_module(out2, *inc1_weights)
        
        max_ = tf.nn.max_pool(out3, ksize=[1, max_pool2_patch, max_pool2_patch, 1], 
                              strides=[1, max_pool2_stride, max_pool2_stride, 1], padding='SAME')
        out4 = run_conv_layer(max_, layer3_weights, layer3_biases)
        
        shape = out4.get_shape().as_list()
        reshape = tf.reshape(out4, [shape[0], shape[1] * shape[2] * shape[3]])
        out5 = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
        out6 = tf.nn.relu(tf.matmul(out5, layer5_weights) + layer5_biases)
        logits = tf.matmul(out6, layer6_weights) + layer6_biases
        return logits
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.MomentumOptimizer(learn_rate, momentum).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 9001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print(' - Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels[:2000]))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.539507
Minibatch accuracy: 11.5%
 - Validation accuracy: 9.9%
